In [278]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from shapely.ops import split
from shapely.geometry import LineString, Point

# Keys used to select traces/boxes by glaciers
GLACIERS = ["DJ", "UI"]

# Reads terminus boxes and traces
terminus_boxes = gpd.read_file("../data/terminus/terminus_boxes.geojson")
terminus_traces = gpd.read_file("../data/terminus/terminus_traces_1985_2023.geojson")

out = []

for glacier in GLACIERS:
    # Subsets data by current glacier
    box = terminus_boxes[terminus_boxes["glacier_code"] == glacier].reset_index()
    traces = terminus_traces[terminus_traces["glacier_code"] == glacier].sort_values(
        by=["date"], ascending=False
    )

    for index, trace in traces.iterrows():
        # Attempts to split box by terminus trace
        difference = split(box.geometry[0], trace.geometry.geoms[0])

        # If split was unsuccessful we extend trace to fully intersect box
        if len(difference.geoms) == 1:
            # Alias necessary geometry
            trace_line = trace.geometry.geoms[0]
            trace_start = trace_line.coords[0]
            trace_end = trace_line.coords[-1]
            box_geom = box.geometry[0].geoms[0]

            # # INITIAL STATE PLOT
            # fig, ax = plt.subplots()
            # x, y = box.geometry[0].geoms[0].exterior.xy
            # plt.plot(x, y, color="blue", alpha=0.5)
            # x, y = trace.geometry.geoms[0].xy
            # plt.plot(x, y, color="orange", alpha=0.5)
            # x, y = trace_start
            # plt.plot(x, y, color="green", marker="o", alpha=0.5)
            # x, y = trace_end
            # plt.plot(x, y, color="red", marker="o", alpha=0.5)
            # plt.title(f"initial: {index}; {trace.date}")
            # plt.show()

            # Trace start point falls within box
            if box_geom.contains(Point(trace_start)):
                # Finds vertex on box edge closest to current start and end points
                nearest_start = box_geom.exterior.interpolate(
                    box_geom.exterior.project(Point(trace_start))
                )
                # Extends relocated start point across box edge to ensure full intersection
                start_edge = box_geom.exterior.interpolate(
                    box_geom.exterior.project(nearest_start) - 50
                )
                start_vec = np.array(nearest_start.coords[0]) - np.array(start_edge.coords[0])
                start_perpendicular = (start_vec[1], -start_vec[0]) / np.linalg.norm(start_vec)
                trace_start_moved = (
                    nearest_start.x - start_perpendicular[0] * 50,
                    nearest_start.y - start_perpendicular[1] * 50,
                )
                # Records distance between trace start and interpolated start
                start_relocate_distance = nearest_start.distance(Point(trace_start))
            else:
                trace_start_moved = trace_start

            # Same process for end point
            if box_geom.contains(Point(trace_end)):
                nearest_end = box_geom.exterior.interpolate(
                    box_geom.exterior.project(Point(trace_end))
                )
                end_edge = box_geom.exterior.interpolate(
                    box_geom.exterior.project(nearest_end) + 50
                )
                end_vec = np.array(nearest_end.coords[0]) - np.array(end_edge.coords[0])
                end_perpendicular = (end_vec[1], -end_vec[0]) / np.linalg.norm(end_vec)
                trace_end_moved = (
                    nearest_end.x + end_perpendicular[0] * 50,
                    nearest_end.y + end_perpendicular[1] * 50,
                )
                end_relocate_distance = nearest_end.distance(Point(trace_end))
            else:
                trace_end_moved = trace_end

            # Reconstructs trace line with new start and/or end coords
            trace_line_extended = list(trace_line.coords)
            trace_line_extended[0] = trace_start_moved
            trace_line_extended[-1] = trace_end_moved
            trace_line_extended = LineString(trace_line_extended)

            # # EXTENDED LINE PLOT
            # fig, ax = plt.subplots()
            # x, y = box.geometry[0].geoms[0].exterior.xy
            # plt.plot(x, y, color="blue", alpha=0.5)
            # x, y = trace_line_extended.xy
            # plt.plot(x, y, color="green", alpha=0.5)
            # plt.title(f"extended: {index}; {trace.date}")
            # plt.show()

            # Splits box by interpolated trace
            difference = split(box.geometry[0], trace_line_extended)

            # # CLOSED BOX PLOT
            # fig, ax = plt.subplots()
            # x, y = difference.geoms[1].exterior.xy
            # plt.plot(x, y, color="blue", alpha=0.5)
            # plt.title(f"difference: {index}; {trace.date}")
            # plt.show()

        # Box already split successfully so assign relocation distances null
        else:
            start_relocate_distance = None
            end_relocate_distance = None

        # Assign attributes from trace to split box
        difference_gdf = trace.drop(labels=["geometry"])
        difference_gdf["geometry"] = difference.geoms[1]
        difference_gdf["area"] = difference.geoms[1].area
        difference_gdf["area_width_averaged"] = difference.geoms[1].area / box.width[0]
        difference_gdf["start_relocated"] = start_relocate_distance
        difference_gdf["end_relocated"] = end_relocate_distance
        difference_gdf = gpd.GeoDataFrame(difference_gdf).transpose().set_geometry("geometry")
        difference_gdf.set_crs(terminus_traces.crs, inplace=True)
        out.append(difference_gdf)

# Construct single DataFrame containing all split boxes
out_gdf = pd.concat(out, ignore_index=True)
out_gdf = out_gdf.sort_values(by=["glacier_code", "date"])

# Calculates change in terminus position relative to earliest observation
out_gdf["area_width_averaged_change"] = out_gdf["area_width_averaged"] - out_gdf.groupby(
    ["glacier_code"]
)["area_width_averaged"].transform("first")
out_gdf["area_width_averaged_change_km"] = (
    out_gdf["area_width_averaged"]
    - out_gdf.groupby(["glacier_code"])["area_width_averaged"].transform("first")
) / 1000

# Exports DataFrame as GeoJSON
out_gdf.to_file("../data/terminus/terminus_boxes_1985_2023.geojson", driver="GeoJSON")

In [313]:
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.dates import DateFormatter, YearLocator
from matplotlib.offsetbox import AnchoredText
from matplotlib.gridspec import GridSpec

GLACIERS = ["DJ", "UI"]
SETTINGS = {
    "DJ": {
        "colour": [(0.122, 0.467, 0.706, 1.0), (0.122, 0.467, 0.706, 0.25)],
        "label": "Daugaard-Jensen",
    },
    "UI": {
        "colour": [(1.0, 0.498, 0.055, 1.0), (1.0, 0.498, 0.055, 0.25)],
        "label": "Uunartit Islands",
    },
}

# Convert date and subset by period
out_gdf["date"] = pd.to_datetime(out_gdf["date"])
data_1985_2010 = out_gdf[(out_gdf["date"] >= "1985-01-01") & (out_gdf["date"] < "2010-01-01")]
data_2010_2023 = out_gdf[(out_gdf["date"] >= "2010-01-01") & (out_gdf["date"] <= "2024-01-01")]

# Global plot settings
plt.rcParams["figure.dpi"] = 1000
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 18
plt.rcParams["axes.labelsize"] = 18
plt.rcParams["xtick.labelsize"] = 18
plt.rcParams["ytick.labelsize"] = 18

# Create figure and axes
# fig, ax = plt.subplots(1, 2, figsize=(17, 6), gridspec_kw={"width_ratios": [1, 2]})

fig = plt.figure(figsize=(16, 6))
gs = GridSpec(nrows=1, ncols=2, width_ratios=[1, 2], height_ratios=[1])
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[0, 1:])
ax = [ax0, ax1]


# Plot data on each subplot
for glacier in GLACIERS:
    glacier_data = data_1985_2010[data_1985_2010["glacier_code"] == glacier]
    ax[0].plot(
        glacier_data["date"],
        glacier_data["area_width_averaged_change_km"],
        label=SETTINGS[glacier]["label"],
        marker="o",
        linewidth=2.5,
        markersize=3,
        markeredgecolor=SETTINGS[glacier]["colour"][0],
        markerfacecolor=SETTINGS[glacier]["colour"][0],
        color=SETTINGS[glacier]["colour"][1],
    )
for glacier in GLACIERS:
    glacier_data = data_2010_2023[data_2010_2023["glacier_code"] == glacier]
    ax[1].plot(
        glacier_data["date"],
        glacier_data["area_width_averaged_change_km"],
        label=SETTINGS[glacier]["label"],
        marker="o",
        linewidth=2.5,
        markersize=3,
        markeredgecolor=SETTINGS[glacier]["colour"][0],
        markerfacecolor=SETTINGS[glacier]["colour"][0],
        color=SETTINGS[glacier]["colour"][1],
    )

# Set y-axis range to fit data and adds label
ax[0].set_ylim(bottom=-10, top=2)
ax[1].set_ylim(bottom=-10, top=2)
ax[0].set_ylabel("Relative terminus position [km]", labelpad=20)

# Set x-axis range to fit data
ax[0].set_xlim(pd.to_datetime("1985-01-01"), pd.to_datetime("2010-01-01"))
ax[1].set_xlim(pd.to_datetime("2010-01-01"), pd.to_datetime("2024-01-01"))

# Set x-axis tick frequency (major)
ax[0].xaxis.set_major_locator(YearLocator(base=5))
ax[0].xaxis.set_major_formatter(DateFormatter("%Y"))
ax[1].xaxis.set_major_formatter(DateFormatter("%Y"))

# Set x-axis tick frequency (minor)
ax[0].xaxis.set_minor_locator(YearLocator(1))
ax[1].xaxis.set_minor_locator(YearLocator(1))

# Rotate x-axis tick labels
ax[0].tick_params(axis="x", rotation=70)
ax[1].tick_params(axis="x", rotation=70)

# Disable y-axis tick labels
ax[1].tick_params(labelleft=False)

# Remove whitespace around data on plot
ax[0].margins(x=0)
ax[1].margins(x=0)

# Draw horizontal line at y=0
ax[0].axhline(y=0, c="black", alpha=0.7, linewidth=0.7, zorder=1, linestyle=(0, (6, 8)))
ax[1].axhline(y=0, c="black", alpha=0.7, linewidth=0.7, zorder=1, linestyle=(0, (6, 8)))

# Add plot labels and advance/retreat text
ax[0].add_artist(
    AnchoredText(
        "(a)",
        loc="upper left",
        borderpad=0.1,
        prop=dict(size=18, family="Times New Roman", weight="semibold", c="black"),
        frameon=False,
    )
)
ax[1].add_artist(
    AnchoredText(
        "(b)",
        loc="upper left",
        borderpad=0.1,
        prop=dict(size=18, family="Times New Roman", weight="semibold", c="black"),
        frameon=False,
    )
)
ax[1].text(0.99, 0.89, "Advance ↑", alpha=0.8, size=18, ha="right", transform=ax[1].transAxes)
ax[1].text(0.99, 0.82, "Retreat ↓", alpha=0.8, size=18, ha="right", transform=ax[1].transAxes)

# Draw legend
ax[0].legend(
    frameon=False,
    loc="lower left",
    fontsize=16,
    markerscale=2,
    scatterpoints=1,
)

# Add gridlines
ax[0].grid(
    True,
    which="major",
    axis="both",
    c="gray",
    linestyle=":",
    linewidth=0.5,
    alpha=0.8,
    zorder=0,
)
ax[1].grid(
    True,
    which="both",
    axis="both",
    c="gray",
    linestyle=":",
    linewidth=0.5,
    alpha=0.8,
    zorder=0,
)

plt.tight_layout()
# plt.subplots_adjust(wspace=0.01)
# plt.savefig(
#     "../img/terminus_boxes_1985_2023_time-series.png",
#     dpi=fig.dpi,
#     bbox_inches="tight",
# )
plt.show()

In [ ]:
import glob

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from matplotlib.dates import DateFormatter, YearLocator, date2num
from matplotlib.font_manager import FontProperties
from matplotlib.gridspec import GridSpec
from matplotlib.offsetbox import AnchoredText
from matplotlib.ticker import FuncFormatter
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from rasterio.plot import show
from rasterio.windows import from_bounds


############################################################################################
## GLOBAL PLOT SETTINGS ####################################################################
############################################################################################
plt.rcParams["figure.dpi"] = 1000
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 18
plt.rcParams["axes.labelsize"] = 27.5  # ~11pt at below figsize
plt.rcParams["xtick.labelsize"] = 25.2  # ~10pt at below figsize
plt.rcParams["ytick.labelsize"] = 25.2


############################################################################################
## CONSTANTS ###############################################################################
############################################################################################
GLACIERS = ["DJ", "UI"]
SETTINGS = {
    "DJ": {
        "colour": [(0.122, 0.467, 0.706, 1.0), (0.122, 0.467, 0.706, 0.25)],
        "label": "Daugaard-Jensen",
        "bbox": [555200, -1897300, 562300, -1890200],
    },
    "UI": {
        "colour": [(1.0, 0.498, 0.055, 1.0), (1.0, 0.498, 0.055, 0.25)],
        "label": "Uunartit Islands",
        "bbox": [489000, -2432000, 502000, -2419000],
    },
}


############################################################################################
## CONSTRUCT PLOT AXES #####################################################################
############################################################################################
fig = plt.figure(figsize=(16, 16))
gs = GridSpec(nrows=2, ncols=2)
ax0 = fig.add_subplot(gs[0, :])
ax1 = fig.add_subplot(gs[1, 0])
ax2 = fig.add_subplot(gs[1, 1])
ax3 = inset_axes(ax0, width="45%", height="30%", loc="lower left", borderpad=2.2)
ax1.set_box_aspect(1)
ax2.set_box_aspect(1)


############################################################################################
## AXIS 0: TERMINUS RETREAT TIME SERIES [FULL] #############################################
############################################################################################
out_gdf["date"] = pd.to_datetime(out_gdf["date"])

for glacier in GLACIERS:
    glacier_subset = out_gdf[out_gdf["glacier_code"] == glacier]
    ax0.plot(
        glacier_subset["date"],
        glacier_subset["area_width_averaged_change_km"],
        label=SETTINGS[glacier]["label"],
        marker="o",
        markersize=3,
        markeredgecolor=SETTINGS[glacier]["colour"][0],
        markerfacecolor=SETTINGS[glacier]["colour"][0],
        linewidth=2.5,
        color=SETTINGS[glacier]["colour"][1],
    )

ax0.set_xlim(left=pd.to_datetime("1985-01-01"), right=pd.to_datetime("2024-01-01"))
ax0.set_ylim(bottom=-10, top=2)
ax0.set_ylabel("Relative terminus position [km]", size=27.5, labelpad=5)
ax0.xaxis.set_minor_locator(YearLocator(1))
ax0.xaxis.set_major_locator(YearLocator(5))
ax0.xaxis.set_major_formatter(DateFormatter("%Y"))
major_ticks = ax0.get_xticks()
major_ticks = np.append(major_ticks, date2num(pd.to_datetime("2024-01-01")))
major_ticks.sort()
ax0.set_xticks(major_ticks[:-1])
ax0.tick_params(axis="x", rotation=45)
ax0.axhline(y=0, c="black", alpha=0.6, linewidth=0.7, zorder=1, linestyle=(0, (6, 8)))
ax0.text(0.95, 0.85, "Advance ↑", alpha=0.8, size=27.5, ha="right", transform=ax0.transAxes)
ax0.text(0.95, 0.79, "Retreat ↓", alpha=0.8, size=27.5, ha="right", transform=ax0.transAxes)
ax0.add_artist(
    AnchoredText(
        "(a)",
        loc="upper left",
        borderpad=0.1,
        prop=dict(size=30, weight="semibold"),
        frameon=False,
    )
)
ax0.legend(
    frameon=False,
    loc="center left",
    fontsize=27.5,
    markerscale=3,
    scatterpoints=1,
    bbox_to_anchor=(0.05, 0.02, 1, 1),
)
ax0.grid(
    True,
    which="major",
    axis="both",
    c="gray",
    linestyle=":",
    linewidth=0.5,
    alpha=0.8,
    zorder=0,
)


############################################################################################
## AXIS 3: TERMINUS RETREAT TIME SERIES [INSET] ############################################
############################################################################################
date_subset = out_gdf[(out_gdf["date"] >= "2010-01-01") & (out_gdf["date"] <= "2024-01-01")]
date_subset["area_width_averaged_change_km_2010_2023"] = (
    date_subset["area_width_averaged"]
    - date_subset.groupby(["glacier_code"])["area_width_averaged"].transform("first")
) / 1000

for glacier in GLACIERS:
    glacier_subset = date_subset[date_subset["glacier_code"] == glacier]
    ax3.plot(
        glacier_subset["date"],
        glacier_subset["area_width_averaged_change_km_2010_2023"],
        label=SETTINGS[glacier]["label"],
        marker="o",
        markersize=1.5,
        markeredgecolor=SETTINGS[glacier]["colour"][0],
        markerfacecolor=SETTINGS[glacier]["colour"][0],
        linewidth=1.75,
        color=SETTINGS[glacier]["colour"][1],
    )

ax3.set_xlim(pd.to_datetime("2010-01-01"), pd.to_datetime("2024-01-01"))
ax3.set_ylim(bottom=-5, top=1)
ax3.xaxis.set_minor_locator(YearLocator(1))
ax3.xaxis.set_major_locator(YearLocator(base=2))
ax3.xaxis.set_major_formatter(DateFormatter("%y"))
ax3.set_yticks([1, 0, -1, -3, -5])
ax3.yaxis.set_minor_locator(plt.MultipleLocator(1))
ax3.tick_params(axis="x", labelsize=22)
ax3.tick_params(axis="y", labelsize=22)
ax3.axhline(y=0, c="black", alpha=0.6, linewidth=0.7, zorder=1, linestyle=(0, (6, 8)))
ax3.grid(
    True,
    which="both",
    axis="both",
    c="gray",
    linestyle=":",
    linewidth=0.5,
    alpha=0.8,
    zorder=0,
)

############################################################################################
## AXIS 1 & 2: TERMINUS POSITIONS MAPS  ####################################################
############################################################################################
basemaps = [rasterio.open(i) for i in glob.glob("../data/misc/imagery/*.tif")[::-1]]
centrelines = gpd.read_file("../data/misc/centreline_lines.geojson")
terminus_boxes = gpd.read_file("../data/terminus/terminus_boxes.geojson")
terminus_traces = gpd.read_file("../data/terminus/terminus_traces_1985_2023.geojson")
terminus_traces["date"] = pd.to_datetime(terminus_traces["date"])

for i, j in enumerate(zip([ax1, ax2], GLACIERS)):
    ax, glacier = j[0], j[1]

    # BASEMAP
    basemap = basemaps[i]
    bbox_window = from_bounds(*SETTINGS[glacier]["bbox"], transform=basemap.transform)
    basemap_raw = basemap.read([3, 2, 1], window=bbox_window)
    basemap_normalised = (basemap_raw * (255 / np.max(basemap_raw))).astype(np.uint8)
    show(basemap_normalised, ax=ax, transform=basemap.window_transform(bbox_window))

    # BOX
    box = terminus_boxes[terminus_boxes["glacier_code"] == glacier].reset_index()
    ax.fill(
        *box.geometry[0].geoms[0].exterior.xy,
        facecolor="white",
        edgecolor="black",
        alpha=0.2,
        linewidth=2,
    )

    # CENTRELINE
    centreline = centrelines[centrelines["glacier_code"] == glacier].reset_index()
    ax.plot(*centreline.geometry[0].xy, c="white", alpha=0.5, linewidth=2)

    # COLOURBAR
    date_nums = date2num(terminus_traces["date"])
    norm = Normalize(vmin=date_nums.min(), vmax=date_nums.max())
    mapper = ScalarMappable(norm=norm, cmap="viridis")

    # TRACES
    traces = terminus_traces[terminus_traces["glacier_code"] == glacier].sort_values(
        by=["date"]
    )
    for index, trace in traces.iterrows():
        colour = mapper.to_rgba(date2num(trace["date"]))
        x, y = trace.geometry.geoms[0].xy
        ax.plot(x, y, c=colour, alpha=0.6)

    # FORMAT
    ax.set_xlim(SETTINGS[glacier]["bbox"][0], SETTINGS[glacier]["bbox"][2])
    ax.set_ylim(SETTINGS[glacier]["bbox"][1], SETTINGS[glacier]["bbox"][3])
    ax.xaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{x/1000:.0f}"))
    ax.yaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{x/1000:.0f}"))
    ax.tick_params(axis="x", labelsize=22)
    ax.tick_params(axis="y", labelsize=22, rotation=90)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.grid(
        True,
        which="both",
        axis="both",
        c="black",
        linestyle=":",
        linewidth=0.5,
        alpha=0.8,
        zorder=0,
    )

ax1.set_ylabel("Terminus position [year]", size=27.5, labelpad=51)

# SCALE BARS
scalebar_font = FontProperties(size=25.2, weight="semibold", math_fontfamily="stix")
ax1.add_artist(
    AnchoredSizeBar(
        ax1.transData,
        2000,
        "2 km",
        "lower right",
        pad=0.8,
        sep=1.25,
        label_top=True,
        color="white",
        frameon=False,
        size_vertical=50,
        fontproperties=scalebar_font,
        fill_bar=True,
    )
)
ax2.add_artist(
    AnchoredSizeBar(
        ax2.transData,
        2000,
        "2 km",
        "lower right",
        pad=0.8,
        sep=1.25,
        label_top=True,
        color="white",
        frameon=False,
        size_vertical=85,
        fontproperties=scalebar_font,
        fill_bar=True,
    )
)

# PLOT LABELS
ax1.add_artist(
    AnchoredText(
        "(b)",
        loc="upper left",
        borderpad=0.1,
        prop=dict(size=30, weight="semibold", c="white"),
        frameon=False,
    )
)
ax2.add_artist(
    AnchoredText(
        "(c)",
        loc="upper left",
        borderpad=0.1,
        prop=dict(size=30, weight="semibold", c="black"),
        frameon=False,
    )
)

# COLOURBAR
min_date = terminus_traces["date"].min()
max_date = terminus_traces["date"].max()
min_year = terminus_traces["date"].dt.year.min()
max_year = terminus_traces["date"].dt.year.max()
first_tick = (min_year // 5 + 1) * 5
year_ticks = np.arange(first_tick, max_year + 1, 5)
year_ticks = np.concatenate(([min_date], year_ticks, [max_date]))
date_ticks = date2num(pd.to_datetime(year_ticks, format="%Y"))
cax = inset_axes(
    ax0,
    width="100%",
    height="5%",
    loc="lower center",
    bbox_to_anchor=(0.0, -1.22, 1, 1),
    bbox_transform=ax0.transAxes,
    borderpad=0,
)
cbar = fig.colorbar(mapper, cax=cax, orientation="horizontal")
cbar.set_ticks(date_ticks)
cbar.ax.tick_params(axis="x", rotation=45, labelsize=25.2)
cbar.ax.xaxis.set_major_formatter(DateFormatter("%Y"))


############################################################################################
## SHOW (AND SAVE)  ########################################################################
############################################################################################
plt.tight_layout()
# plt.savefig(
#     "../data/terminus/terminus_boxes_1985_2023_complete.png",
#     dpi=fig.dpi,
#     bbox_inches="tight",
# )
plt.show()